# Example 4: Ternary Tangent Pinch

#### Example 3 depicted a tangent pinch for a binary system.  This example shows how a tangent pinch is represented in a ternary system of Acetaldehyde, Methanol, and Water.

In these systems, as Reflux Ratio changes, the fixed point which terminates the rectifying section changes.  However, at the tangent pinch, a very small increment in reflux ratio causes the fixed point composition to rapidly shoot to a new value. 

This example shows the rectifying section experiences a major jump at this tangent pinch, which occurs at a reflux ratio between 0.1 and 0.2.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display
from src.thermo_models.RaoultsLawModel import *
from src.distillation.DistillationModel import DistillationModel
from src.utils.AntoineEquation import * 
from IPython.display import clear_output
from tqdm import tqdm
from src.distillation.DistillationTernary import *
import ipywidgets as widgets
from IPython.display import display
from ipywidgets import interactive, Button, VBox, HBox, Output
from IPython.display import clear_output
from src.thermo_models.MargulesModel import *
from src.thermo_models.WilsonModel import *


###### System parameters using Margules Model

In [2]:
A_ = np.array([[0, -316.699, 350.100], [-384.657, 0, 307.000],[290.200, 143.00, 0]])

#Example 4 in Fidkowski is at 1 atm = 760 mmHg
P_sys = 1.01325 #bar

#Antoine parameters from table 2.1 in page 25 of Doherty Malone Distillation Textbook
#Units: P_sat in mmHg and Temperature in Celsius 
#Form is P_sat = A - B/(T+C)

#Antoine parameters from NIST (Kelvin, Bar)
#Acetaldehyde
Acet_A = 3.68639
Acet_B = 822.894
Acet_C = -69.899
#Methanol
Me_A = 5.31301
Me_B = 1676.569
Me_C = -21.728
#Water
H2O_A = 3.55959
H2O_B = 643.748
H2O_C = -198.043

#Antoine Equations 
Acet_antoine = AE.AntoineEquationBase10(Acet_A, Acet_B, Acet_C)
H2O_antoine = AE.AntoineEquationBase10(H2O_A, H2O_B, H2O_C)
Methanol_antoine = AE.AntoineEquationBase10(Me_A, Me_B, Me_C)

vle_model = MargulesModel(3,1,A_,["Acet", "Meth","H2O"],[Acet_antoine, Methanol_antoine, H2O_antoine], True)

In [3]:
R1_slider =  widgets.FloatSlider(value=0.1, min=0, max=1, step=0.1,  description='Reflux Ratio:', readout_format='.1f')
R2_slider =  widgets.FloatSlider(value=0.1, min=0, max=1, step=0.1,  description='Reflux Ratio:', readout_format='.1f')
go_button = widgets.Button(description='Create Model', width='15%', margin='0 10px 0 0')
out_1 = Output()
out_2 = Output()

def plot_rect(R = R1_slider):
    plt.close('all')  # Ensure that any existing figures are closed before plotting a new one.
    xF = np.array([0.3, 0.3, 0.4])
    xD = np.array([0.5030, 0.4960, 0.0010])
    xB = np.array([1e-10, 0.0103,0.9897])
    distillation_model = DistillationModelTernary(vle_model, xF = xF, xD = xD, xB = xB, reflux = R)
    fig, ax = plt.subplots(1,1,figsize= (5,5))
    distillation_model.plot_rect_comp(ax)
    plt.subplots_adjust(hspace=0)
    plt.show()

def plot_rect_strip(R = R2_slider):
    plt.close('all')  # Ensure that any existing figures are closed before plotting a new one.
    xF = np.array([0.3, 0.3, 0.4])
    xD = np.array([0.5030, 0.4960, 0.0010])
    xB = np.array([1e-10, 0.0103,0.9897])
    distillation_model = DistillationModelTernary(vle_model, xF = xF, xD = xD, xB = xB, reflux = R)
    fig, ax = plt.subplots(1,1,figsize= (5,5))
    distillation_model.plot_rect_strip_comp(ax)
    
    # for index,point in enumerate(fixed_points['values']):
    #     ax.scatter(point[0], point[1], marker='x', color='black', s=100, linewidths= 10)
    #     if fixed_points['source_indices'][index] == 0:
    #         label = "Unstable"
    #     elif fixed_points['source_indices'][index] == 1:
    #         label = "saddle"
    #     elif fixed_points['source_indices'][index] == 2:
    #         label = "stable"
    #     else:
    #         label = "unexpected node: error"
        # ax.text(point[0], point[1], f' {label}', verticalalignment='bottom', horizontalalignment='right')
    plt.subplots_adjust(hspace=0)
    plt.show()

### Demo 1: Rectifying Section

Use the slider to vary the reflux ratio.  Notice how the end of the rectifying section always changes with the reflux ratio, but experiences a much more rapid change between values of 0.1 and 0.2.

In [4]:
def on_go_button_click(b):
    with out_1:
        clear_output(wait=True)
        plt.close('all')  # Close all open matplotlib plots
        plot_rect( R=R1_slider.value)
        
go_button = Button(description='Generate Plot')
go_button.on_click(on_go_button_click)

layout = VBox([HBox([R1_slider, go_button]), out_1])
display(layout)

### Demo 2: Full Column

What are the implications of this ternary tangent pinch?

As Example 2 discussed, for a ternary column to be feasible, the rectifying and stripping sections need to overlap.
Change the reflux ratio again, and notice how the jump at the ternary pinch value is necessary for the column to be feasible.  This demonstration also highlights how slight changes to reflux ratio have a much larger impact on the rectifying section than on the stripping section.

In [5]:

def on_go2_button_click(b):
    with out_2:
        clear_output(wait=True)
        plt.close('all')  # Close all open matplotlib plots
        plot_rect_strip( R=R2_slider.value)
        
go2_button = widgets.Button(description='Generate Plot')
go2_button.on_click(on_go2_button_click)

layout = VBox([HBox([R2_slider, go2_button]), out_2])
display(layout)